In [1]:
import sys; sys.path.insert(0, "../")
import numpy as np
from models import *
from pytorch_robust_pca import *
from utils import *

Running Python 3.9.6
You can use npar for np.array


In [2]:
import numpy as np

# generate low rank synthetic data
N = 100
num_groups = 3
num_values_per_group = 40
p_missing = 0.2

Ds = []
for k in range(num_groups):
    d = np.ones((N, num_values_per_group)) * (k + 1) * 10
    Ds.append(d)

D = np.hstack(Ds)

# decimate 20% of data
n1, n2 = D.shape
S = np.random.rand(n1, n2)
D[S < 0.2] = 0

D = torch.FloatTensor(D).requires_grad_(False)

In [3]:
model = RobustPCANeuralNet(input_tensor=D)

The settings are ...
Lambda: tensor(0.0913)
Inverse mu: tensor(0.9100)


In [4]:
max_iter = 10000
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
for i in range(max_iter):
    opt.zero_grad()
    L = model(D)
    l = model.loss(D, L)
    l.backward()
    if model.is_terminated(): opt.zero_grad(); break
    opt.step()
    if i%1000==0: print(l.item())

0.09817618131637573
0.036400508135557175
0.014272257685661316
0.013292652554810047
0.012357961386442184


In [5]:
((D-(L+model.S))**2).mean()

tensor(0., grad_fn=<MeanBackward0>)

In [6]:
model.S

Parameter containing:
tensor([[  4.7937,   4.7484,   4.8073,  ...,  15.1697,  14.8914,  15.1446],
        [ -4.5126,  -4.5513,   5.4949,  ...,  17.1391,  16.8941,  17.1142],
        [  5.3334,   5.2933,   5.3438,  ...,  16.7049,  16.4526,  16.6775],
        ...,
        [ -5.0594,   4.8938,  -5.0482,  ...,  15.5851,  15.3118, -14.4428],
        [  5.4552,   5.4156,   5.4651,  ..., -12.9520,  16.8043,  17.0239],
        [  5.1794,  -4.8622,   5.1906,  ...,  16.2693,  16.0088, -13.7603]],
       requires_grad=True)

In [7]:
print(torch.linalg.matrix_rank(L).item())
print(max(L.shape))

37
120


In [8]:
torch.linalg.matrix_rank(L).item()/max(L.shape)

0.30833333333333335

In [9]:
rpca = R_pca(D)
L, S = rpca(max_iter=10000, iter_print=100)
print('MSE Loss:', F.mse_loss(D, L+S).item())

MSE Loss: 2.925412342924538e-11


In [10]:
np.linalg.matrix_rank(L)

1